In [44]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras import backend as K



In [8]:
ls ../data

country_code_lookup.csv  yemen_batch0.pkl  yemen_batch1.pkl
event_code_lookup.csv    yemen_batch1.csv


In [46]:
yemen_df0 = pd.read_csv('../data/yemen_batch0.csv', index_col=0)

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.103250,False,0,9,0.000000
2014-02-19,YEMENI,LIBYA,1,1.9,2,-6.102747,False,0,9,0.000000
2014-02-19,LIBYA,YEMENI,1,2.8,2,-6.102747,False,0,9,0.000000
2014-02-19,YEMENI,LIBYA,1,1.9,1,-6.243603,False,0,9,0.000000
2014-02-19,NaN,ARMY,1,8.0,10,-2.422907,False,0,9,0.000000
2014-02-19,LIBYA,YEMENI,1,2.8,1,-6.243603,False,0,9,0.000000
2014-02-19,CLERIC,LIBYA,1,2.8,2,-3.781175,False,0,9,0.000000
2014-02-19,INTELLIGENCE SERVICE,DANISH,1,0.0,2,-9.411765,False,0,9,0.000000
2014-02-19,INTELLIGENCE SERVICE,DANISH,1,0.0,6,-9.411765,False,0,9,0.000000
